In [1]:
import pandas as pd, numpy as np
from keybert._model import KeyBERT
import nltk, re
from nltk.corpus import stopwords

In [2]:
tenders_info = pd.read_csv('./Data/tenders_info.csv')

In [3]:
tenders_info['text'] = tenders_info['Description'] + '.' + tenders_info['Title']

In [4]:
STOP_WORDS = stopwords.words('english')
kw_model = KeyBERT()

In [5]:
def __get_tags(text):
    keywords = kw_model.extract_keywords(text,
                                                  keyphrase_ngram_range=(1, 3),
                                                  stop_words='english',
                                                  use_mmr=True,
                                                  diversity=0.2)
    if len(keywords) == 0:
            keywords = kw_model.extract_keywords(text, keyphrase_ngram_range=(1, 3), stop_words='english')
    return keywords if len(keywords) > 0 else [('[none_tag]', 1)]
def __agg_tags(self, text) -> pd.DataFrame:
    reg_rule = r'\((.*?)\)'
    text = str(text)

    # Extract and reformat term-weight set
    
    split_result = tmp_df[0].str.split(',', expand=True).rename(columns={0: 'key',
                                                                             1: 'value'}).reset_index()

    merge_df = tmp_df.merge(split_result, on='index').drop('index', axis=1).rename(columns={'level_0': 'items'})
    del tmp_df, split_result

    merge_df['key'] = merge_df['key'].map(self.__split_words)
    mapping_df = merge_df.explode('key')[['items', 'key', 'value']]
    mapping_df['key_orig'] = mapping_df['key']
    # mapping_df['key'] = mapping_df['key'].map(nltk.PorterStemmer().stem)

    # Compute weight according to each word and ordering
    mapping_df['value'] = mapping_df['value'].astype(float)
    sum_df = mapping_df.groupby(['items', 'key', 'key_orig'])['value'].sum().reset_index().sort_values(
            ['items', 'value'], ascending=False)

    # Sampling top key
    key_df = sum_df.groupby(['items']).head(1)[['items', 'key']]
    merge_df = sum_df.merge(key_df, on=['items', 'key'])
    return merge_df
def __convert_word_type(word):
    token = nltk.word_tokenize(word.lower())
    lemmatizer = nltk.stem.WordNetLemmatizer()
    pos_tagged = nltk.pos_tag(token)
    words = []
    Noun = list(filter(
        lambda x: x[0] not in STOP_WORDS and x[0] != '/' and (x[1].startswith('NN') or x[1].startswith('JJ')),
            pos_tagged))
    for word, pos in Noun:
        if pos.startswith('NN'):
            word = lemmatizer.lemmatize(word, pos='n')
        elif pos.startswith('JJ'):
            word = lemmatizer.lemmatize(word, pos='a')
        words.append(word)
    text = " ".join(words)
    return text

In [6]:
def __extract_process(index, row):
    text = row['text']
    if len(text) == 0: return []
    result_list = __get_tags(text)
    print(result_list)
    return index+4
    

In [7]:
df = tenders_info.copy()[:10]
df['text'].fillna('', inplace=True)
assert 'text' in df.columns, f'Missing column names "{text_col}".'

df['text'] = df['text'].map(lambda x: __convert_word_type(x))

# Remove pure numbers, e.g., 2014, 20,000
df['text'] = df['text'].map(lambda x: re.sub(r'\s*(\.:,|\d+)\s*', '', x))
df = df.reset_index(drop=True).reset_index()
df

,index,_id,Agency,Title,Publish Date,Close Date,Category,Category_sub,Description,Eligibility,Value,Location,text
0,0,6233feb7c546c803a643a750,"Department of Agriculture, Water and the Envir...",Australia-China Agricultural Cooperation Agree...,2017-02-10,2017-03-10,Overseas Advocacy,NaN,The Australia–China Agricultural Cooperation A...,"Any individual, organisation or business with ...",NaN,"ACT, NSW, VIC, SA, WA, QLD, NT, TAS",australia–china agricultural cooperation agree...
1,1,6233febac546c803a643a751,Department of Foreign Affairs and Trade,Australia-ASEAN Council 2017 Grant Round,2017-02-06,2017-03-27,Public Diplomacy,NaN,The Australia-ASEAN Council's (AAC’s) mission ...,To be eligible you must: be one of the followi...,1000000-5000000,"ACT, NSW, VIC, SA, WA, QLD, NT, TAS, Overseas",australia-asean council aac ’ mission knowledg...
2,2,6233febcc546c803a643a752,"Department of Agriculture, Water and the Envir...",Package Assisting Small Exporters (PASE) Round 2,2017-02-24,2017-03-27,Small Business,341000 - Trade and Tourism,"In the 2014-15 budget, the government committe...",The following can apply for a grant: - Organis...,NaN,"ACT, NSW, VIC, SA, WA, QLD, NT, TAS",-budget government year package small exporter...
3,3,6233febec546c803a643a753,Department of Foreign Affairs and Trade,Council on Australia Latin America Relations,2017-02-06,2017-03-28,Public Diplomacy,NaN,The Council on Australia Latin America Relatio...,To be eligible you must: be one of the followi...,2000000-4000000,"ACT, NSW, VIC, SA, WA, QLD, NT, TAS, Overseas",council australia latin america relation coala...
4,4,6233fec0c546c803a643a754,Department of Health,National Aged Care Advocacy Program (NACAP) Fu...,2017-02-16,2017-03-31,Aged Care,331001 - Advocacy,The National Aged Care Advocacy Program provid...,The following entity types meet the eligibilit...,NaN,"ACT, NSW, VIC, SA, WA, QLD, NT, TAS",national care advocacy program free independen...
5,5,6233fec3c546c803a643a755,Department of Foreign Affairs and Trade,Australia-Korea Foundation 2017 Grant Round,2017-02-06,2017-04-03,Public Diplomacy,NaN,The Australia-Korea Foundation 2017 seeks to s...,To be eligible you must: be one of the follow...,1500000-3000000,"ACT, NSW, VIC, SA, WA, QLD, NT, TAS, Overseas",australia-korea foundation seek australia-kore...
6,6,6233fec5c546c803a643a756,Department of Foreign Affairs and Trade,Council for Australian-Arab Relations,2017-02-06,2017-04-03,Public Diplomacy,NaN,The Council for Australian-Arab Relations seek...,To be eligible you must: be one of the followi...,2000000-4000000,"ACT, NSW, VIC, SA, WA, QLD, NT, TAS, Overseas",council australian-arab relation australian-ar...
7,7,6233fec7c546c803a643a757,Department of Foreign Affairs and Trade,Australia-Japan Foundation 2017 Grant Round,2017-02-06,2017-04-06,Public Diplomacy,NaN,The Australia-Japan Foundation seeks to advanc...,To be eligible you must: be one of the follow...,500000-5000000,"ACT, NSW, VIC, SA, WA, QLD, NT, TAS, Overseas",australia-japan foundation australia ’ engagem...
8,8,6233fec9c546c803a643a758,Department of Foreign Affairs and Trade,Australian Cultural Diplomacy Grants Program 2...,2017-02-06,2017-04-12,Public Diplomacy,NaN,The Australian Cultural Diplomacy Grants Progr...,To be eligible you must: be one of the followi...,500000-6000000,"ACT, NSW, VIC, SA, WA, QLD, NT, TAS, Overseas",australian cultural diplomacy grant program ac...
9,9,6233feccc546c803a643a759,Department of the Prime Minister and Cabinet,National Women's Alliances,2017-03-16,2017-04-17,Women,NaN,The National Women’s Alliance Grant Opportunit...,We can only accept applications from: existin...,NaN,"ACT, NSW, VIC, SA, WA, QLD, NT, TAS",national woman alliance grant opportunity form...


In [53]:
for i in range(len(df)):
    __extract_process(thread_cnt[0], df.iloc[i])

[('cooperation agriculture australia', 0.6811), ('australia china programme', 0.6411), ('acaca agreement programme', 0.5605), ('agriculture australia china', 0.6425), ('china agricultural cooperation', 0.6282)]
[('education australia asean', 0.6942), ('australia asean council', 0.6926), ('australia asean', 0.6915), ('knowledge understanding australia', 0.6042), ('country opportunity awareness', 0.5544)]


In [51]:
import threading
class KeyThread(threading.Thread):
    
    def __init__(self,func, args=()):
        super(MyThread,self).__init__()
        self.func = func
        self.args = args
        self.result = None

    def run(self):
        self.result = self.func(*self.args)

    def get_result(self):
        if self.result:
            return self.result
        else:
            self.join()
            return self.result

In [50]:
thread_cnt = [0,1,2,3]
while max(thread_cnt) < len(df):
    threads = []
    t1 = KeyThread(__extract_process, args=(thread_cnt[0], df.iloc[thread_cnt[0]]))
    t2 = KeyThread(__extract_process, args=(thread_cnt[1], df.iloc[thread_cnt[1]]))
    t3 = KeyThread(__extract_process, args=(thread_cnt[2], df.iloc[thread_cnt[2]]))
    t4 = KeyThread(__extract_process, args=(thread_cnt[3], df.iloc[thread_cnt[3]]))
    threads.append(t1)
    threads.append(t2)
    threads.append(t3)
    threads.append(t4)

    for t in threads:
        t.setDaemon(True)
        t.start()
        
    for i in range(4):
        threads[i].join()
        thread_cnt[i] = threads[i].get_result()
    print(thread_cnt)

Exception in thread Thread-231:
Traceback (most recent call last):
  File "c:\users\24966\appdata\local\programs\python\python39\lib\threading.py", line 950, in _bootstrap_inner
    self.run()
  File "C:\Users\24966\AppData\Local\Temp/ipykernel_14276/856998351.py", line 11, in run
  File "C:\Users\24966\AppData\Local\Temp/ipykernel_14276/324543592.py", line 4, in __extract_process
  File "C:\Users\24966\AppData\Local\Temp/ipykernel_14276/1638120703.py", line 2, in __get_tags
  File "c:\users\24966\appdata\local\programs\python\python39\lib\site-packages\keybert\_model.py", line 113, in extract_keywords
    keywords = self._extract_keywords_single_doc(doc=docs,
  File "c:\users\24966\appdata\local\programs\python\python39\lib\site-packages\keybert\_model.py", line 182, in _extract_keywords_single_doc
    doc_embedding = self.model.embed([doc])
  File "c:\users\24966\appdata\local\programs\python\python39\lib\site-packages\keybert\backend\_sentencetransformers.py", line 53, in embed
    

[('education australia asean', 0.6942), ('australia asean council', 0.6926), ('australia asean', 0.6915), ('knowledge understanding australia', 0.6042), ('country opportunity awareness', 0.5544)]
[('cooperation agriculture australia', 0.6811), ('australia china programme', 0.6411), ('acaca agreement programme', 0.5605), ('agriculture australia china', 0.6425), ('china agricultural cooperation', 0.6282)]
[4, 5, None, None]


TypeError: '>' not supported between instances of 'NoneType' and 'int'

In [29]:
thread_cnt = [0,1,2,3]
threads = []
t1 = MyThread(__extract_process, args=(thread_cnt[0], df.iloc[thread_cnt[0]]))
t2 = MyThread(__extract_process, args=(thread_cnt[1], df.iloc[thread_cnt[1]]))
t3 = MyThread(__extract_process, args=(thread_cnt[2], df.iloc[thread_cnt[2]]))
t4 = MyThread(__extract_process, args=(thread_cnt[3], df.iloc[thread_cnt[3]]))
threads.append(t1)
threads.append(t2)
threads.append(t3)
threads.append(t4)

for t in threads:
    t.setDaemon(True)  # 设置为守护线程，不会因主线程结束而中断
    t.start()
        
for i in range(4):
    threads[i].join()
    thread_cnt[i] = threads[i].get_result()
print(thread_cnt)

Exception in thread Thread-36:
Traceback (most recent call last):
  File "c:\users\24966\appdata\local\programs\python\python39\lib\threading.py", line 950, in _bootstrap_inner
    self.run()
  File "C:\Users\24966\AppData\Local\Temp/ipykernel_14276/2673266896.py", line 10, in run
  File "C:\Users\24966\AppData\Local\Temp/ipykernel_14276/324543592.py", line 4, in __extract_process
  File "C:\Users\24966\AppData\Local\Temp/ipykernel_14276/1638120703.py", line 2, in __get_tags
  File "c:\users\24966\appdata\local\programs\python\python39\lib\site-packages\keybert\_model.py", line 113, in extract_keywords
    keywords = self._extract_keywords_single_doc(doc=docs,
  File "c:\users\24966\appdata\local\programs\python\python39\lib\site-packages\keybert\_model.py", line 182, in _extract_keywords_single_doc
    doc_embedding = self.model.embed([doc])
  File "c:\users\24966\appdata\local\programs\python\python39\lib\site-packages\keybert\backend\_sentencetransformers.py", line 53, in embed
    

[('cooperation agriculture australia', 0.6811), ('australia china programme', 0.6411), ('acaca agreement programme', 0.5605), ('agriculture australia china', 0.6425), ('china agricultural cooperation', 0.6282)]
[4, None, None, None]


In [34]:
col_pool

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]